# Courses Demo
This Jupyter notebook is for exploring the data set courses20-21.json
which consists of all Brandeis courses in the 20-21 academic year (Fall20, Spr21, Sum21) 
which had at least 1 student enrolled.

First we need to read the json file into a list of Python dictionaries

In [1]:
import statistics

In [2]:
import json

In [3]:
with open("courses20-21.json","r",encoding='utf-8') as jsonfile:
    courses = json.load(jsonfile)

## Structure of a course
Next we look at the fields of each course dictionary and their values

In [4]:
print('there are',len(courses),'courses in the dataset')
print('here is the data for course 1246')
courses[1246]

there are 7813 courses in the dataset
here is the data for course 1246


{'limit': 28,
 'times': [{'start': 1080, 'end': 1170, 'days': ['w', 'm']}],
 'enrolled': 4,
 'details': 'Instruction for this course will be offered remotely. Meeting times for this course are listed in the schedule of classes (in ET).',
 'type': 'section',
 'status_text': 'Open',
 'section': '1',
 'waiting': 0,
 'instructor': ['An', 'Huang', 'anhuang@brandeis.edu'],
 'coinstructors': [],
 'code': ['MATH', '223A'],
 'subject': 'MATH',
 'coursenum': '223A',
 'name': 'Lie Algebras: Representation Theory',
 'independent_study': False,
 'term': '1203',
 'description': "Theorems of Engel and Lie. Semisimple Lie algebras, Cartan's criterion. Universal enveloping algebras, PBW theorem, Serre's construction. Representation theory. Other topics as time permits. Usually offered every second year.\nAn Huang"}

## Cleaning the data
If we want to sort courses by instructor or by code, we need to replace the lists with tuples (which are immutable lists)

In [5]:
for course in courses:
        course['instructor'] = tuple(course['instructor'])
        course['coinstructors'] = tuple([tuple(f) for f in course['coinstructors']])
        course['code']= tuple(course['code'])

# Exploring the data set
Now we will show how to use straight python to explore the data set and answer some interesting questions. Next week we will start learning Pandas/Numpy which are packages that make it easier to explore large dataset efficiently.

Here are some questions we can try to asnwer:
* what are all of the subjects of courses (e.g. COSI, MATH, JAPN, PHIL, ...)
* which terms are represented?
* how many instructors taught at Brandeis last year?
* what were the five largest course sections?
* what were the five largest courses (where we combine sections)?
* which are the five largest subjects measured by number of courses offered?
* which are the five largest courses measured by number of students taught?
* which course had the most sections taught in 20-21?
* who are the top five faculty in terms of number of students taught?
* etc.

#### a. How many faculty taught COSI courses last year?

In [6]:
len({course['instructor'] for course in courses if course['subject']=='COSI' and (course['term']=='1211' or course['term'] == '1212')})

27

#### b. what is the total number of students taking COSI courses last year?

In [7]:
sum({course['enrolled'] for course in courses if course['subject'] == 'COSI' and (course['term']=='1211' or course['term'] == '1212')})

944

#### c. what was the median size of a COSI course last year (counting only those courses with at least 10 students)

In [8]:
statistics.median({course['enrolled'] for course in courses if course['subject'] == 'COSI' and 
                   (course['term']=='1211' or course['term'] == '1212') and (course['enrolled']>=10)})

35.0

#### d. create a list of tuples (E,S) where S is a subject and E is the number of students enrolled in courses in that subject, sort it and print the top 10. This shows the top 10 subjects in terms of number of students taught.


In [9]:
tup = dict()
for c in courses:
    if c['subject'] in tup:
        tup[c['subject']] += c['enrolled']
    else:
        tup[c['subject']] = c['enrolled']

list = [(k, v) for k, v in tup.items()]
list = sorted(list, key = lambda x: -x[1])
list[0: 10]

# [ins for ins in sorted(tup.items(), key= lambda tup: tup[1], reverse = True)][:10]
# data = list(zip(A, B))
# data = sorted(data, key = lambda x: x[1])
# data[0:10]



[('HS', 5318),
 ('BIOL', 3085),
 ('BUS', 2766),
 ('HWL', 2734),
 ('CHEM', 2322),
 ('ECON', 2315),
 ('COSI', 2223),
 ('MATH', 1785),
 ('PSYC', 1704),
 ('ANTH', 1144)]

#### e. do the same as in (d) but print the top 10 subjects in terms of number of courses offered


In [10]:
tup1 = dict()
for c in courses:
    if c['subject'] in tup1:
        tup1[c['subject']] += 1
    else:
        tup1[c['subject']] = 1

list1 = [(k, v) for k, v in tup1.items()]
list1 = sorted(list1, key = lambda x: -x[1])
list1[0: 10]

# not sure about what it mean by number of courses: same course different term?


[('BIOL', 613),
 ('HIST', 498),
 ('PSYC', 417),
 ('NEUR', 403),
 ('BCHM', 296),
 ('PHYS', 288),
 ('HS', 274),
 ('COSI', 272),
 ('MUS', 266),
 ('ENG', 265)]

#### f. do the same as (d) but print the top 10 subjects in terms of number of faculty teaching courses in that subject


In [11]:
tup2 = dict()
for c in courses:
    if c['subject'] in tup2:
        if c['instructor'] in tup2[c['subject']]:
            continue
        else:
            tup2[c['subject']].add(c['instructor'])
    else:
        tup2[c['subject']] = set()
        tup2[c['subject']].add(c['instructor'])

list2 = [(k, len(v)) for k, v in tup2.items()]
list2 = sorted(list2, key = lambda x: -x[1])
list2[0: 10]

[('HS', 87),
 ('BIOL', 67),
 ('ECON', 52),
 ('BCHM', 49),
 ('BUS', 47),
 ('HIST', 47),
 ('BCBP', 46),
 ('HWL', 42),
 ('MATH', 37),
 ('NEJS', 37)]

#### g. list the top 20 faculty in terms of number of students they taught

In [12]:
instructor = dict()
for c in courses:
    if c['instructor'] in instructor:
        instructor[c['instructor']] += c['enrolled']
    else:
        instructor[c['instructor']] = c['enrolled']
[ins for ins in sorted(instructor.items(), key= lambda instructor: instructor[1], reverse = True)][:10]

[(('Leah', 'Berkenwald', 'leahb@brandeis.edu'), 926),
 (('Kene Nathan', 'Piasta', 'kpiasta@brandeis.edu'), 583),
 (('Stephanie', 'Murray', 'murray@brandeis.edu'), 515),
 (('Milos', 'Dolnik', 'dolnik@brandeis.edu'), 489),
 (('Maria', 'de Boef Miara', 'mmiara@brandeis.edu'), 450),
 (('Bryan', 'Ingoglia', 'ingoglia@brandeis.edu'), 439),
 (('Rachel V.E.', 'Woodruff', 'woodruff@brandeis.edu'), 422),
 (('Timothy J', 'Hickey', 'tjhickey@brandeis.edu'), 411),
 (('Daniel', 'Breen', 'dbreen91@brandeis.edu'), 375),
 (('Melissa', 'Kosinski-Collins', 'kosinski@brandeis.edu'), 365)]

#### h. list the top 20 courses in terms of number of students taking that course (where you combine different sections and semesters, i.e. just use the subject and course number)

In [13]:
courses_students = dict()
for c in courses:
    if c['code'] in courses_students.keys():
        courses_students[c['code']] += c['enrolled']
    else:
        courses_students[c['code']] = 0
courses_students = sorted(courses_students.items(), key = lambda item: item[1], reverse = True)
print(courses_students[:20])

[(('HWL', '1'), 915), (('HWL', '1-PRE'), 784), (('BIOL', '14A'), 316), (('MATH', '10A'), 255), (('BIOL', '18B'), 251), (('BIOL', '18A'), 210), (('COSI', '10A'), 193), (('CHEM', '29A'), 193), (('COSI', '12B'), 182), (('BUS', '6A'), 178), (('CHEM', '29B'), 171), (('PSYC', '10A'), 166), (('BUS', '10A'), 163), (('MATH', '15A'), 161), (('EL', '35A'), 161), (('ECON', '80A'), 150), (('ECON', '10A'), 138), (('CHEM', '11B'), 137), (('CHEM', '18B'), 137), (('CHEM', '18A'), 137)]


#### i. Create my own interesting question(Lu):
    What are the courses that mentioned code in its description?

In [ ]:
{c['name'] for c in courses if 'code' in c['description']}

#### i. Create my own interesting question(Jing):
    The top 10 instructors in terms of the number of independent study they have taught

In [ ]:
instructor_with_inde = dict()
for c in courses:
    if course['independent_study'] == True:
        if c['instructor'] in instructor_with_inde:
            instructor_with_inde[c['instructor']] += 1
        else:
            instructor_with_inde[c['instructor']] = 1
[ins for ins in sorted(instructor_with_inde.items(), key= lambda instructor: instructor[1], reverse = True)][:10]

#### i. Create my own interesting question(Shuyuan):
    Courses waiting list is larger than 15?

In [14]:
[{c['name'] for c in courses if c['waiting'] >= 15}] 

[{'Biomedical Ethics',
  'Biostatistics',
  'Darwinian Dating: The Evolution of Human Attraction',
  'Human Anatomy',
  'Introduction to Cognitive Neuroscience',
  'Introduction to Probability and Statistics',
  'Money Management for Beginners',
  'Personal Safety/Self-Defense',
  'Yoga'}]